In [21]:
from IPython.display import Audio, display
import torch

from api import StableTTSAPI

device = 'cuda' if torch.cuda.is_available() else 'cpu'

tts_model_path = './checkpoints/checkpoint_step_2457.pt' # path to StableTTS checkpoint
vocoder_model_path = './vocoders/pretrained/firefly-gan-base-generator.ckpt' # path to vocoder checkpoint
vocoder_type = 'ffgan' # ffgan or vocos

# vocoder_model_path = './vocoders/pretrained/vocos.pt'
# vocoder_type = 'vocos'

model = StableTTSAPI(tts_model_path, vocoder_model_path, vocoder_type)
model.to(device)

tts_param, vocoder_param = model.get_params()
print(f'tts_param: {tts_param}, vocoder_param: {vocoder_param}')

tts_param: 31.578753, vocoder_param: 36.51133


In [22]:
# text = 'ស្ពាន កំពង់ ចម្លង អ្នកលឿង នៅ ព្រៃវែង ជា ស្ពាន វែង ជាង គេ ក្នុង ព្រះ រាជាណាចក្រ កម្ពុជា' 
text = 'ស្ពាន កំពង់ ចម្លង អ្នកលឿង នៅ ព្រៃវែង ជា ស្ពាន វែង ជាង គេ ក្នុង ព្រះ រាជាណាចក្រ កម្ពុជា'
ref_audio = '/Users/nguyenthithutam/StableTTS/raw_datasets/khmer/wavs/khm_0308_0011865648.wav'
language = 'khmer' # support chinese, japanese and english
solver = 'dopri5' # recommend using euler, midpoint or dopri5
steps = 30
cfg = 3 # recommend 1-4

audio_output, mel_output = model.inference(text, ref_audio, language, steps, 1, 1, solver, cfg)

display(Audio(ref_audio))
display(Audio(audio_output, rate=model.mel_config.sample_rate))

[INFO] Loaded audio via soundfile: /Users/nguyenthithutam/StableTTS/raw_datasets/khmer/wavs/khm_0308_0011865648.wav, sr=48000, shape=torch.Size([1, 335872])


In [15]:
!pip install torchcodec


In [20]:
!conda install "ffmpeg<8"

Channels:
 - defaults
 - conda-forge
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3/envs/stabletts

  added / updated specs:
    - ffmpeg[version='<8']


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    aom-3.12.1                 |       ha237518_0         2.4 MB
    ca-certificates-2025.11.4  |       hca03da5_0         128 KB
    ffmpeg-6.1.1               |       h191590f_4         8.1 MB
    fontconfig-2.15.0          |       h29935d0_0         225 KB
    freetype-2.13.3            |       h47d26ad_0         626 KB
    fribidi-1.0.10             |       h1a28f6b_0          60 KB
    gdk-pixbuf-2.42.10         |       h969f7ef_2         579 KB
    gettext-0.21.0             |       hbdbcc25_2         3.1 MB
    giflib-5.2.2               |       h80987f9_0          78 KB
    glib-2.84.4                |       h863a760_0         

In [7]:
pip install torchcodec==0.2.1


Note: you may need to restart the kernel to use updated packages.


In [8]:
print(model.g2p_mapping.keys())


dict_keys(['chinese', 'japanese', 'english', 'khmer'])


In [11]:
import re

lexicon_path = "text/lexicons/khmer.tsv"

phones = set()

with open(lexicon_path, "r", encoding="utf-8") as f:
    for line in f:
        if not line.strip() or line.startswith("#"):
            continue
        parts = line.strip().split("\t")
        if len(parts) < 2:
            continue
        phonemes = parts[1].split()
        for ph in phonemes:
            phones.add(ph)

# Sắp xếp để xem rõ
phones = sorted(list(phones))
print(f"Total phonemes: {len(phones)}")
print(" ".join(phones))


Total phonemes: 55
. a aa ae ao aə c ch e ea ee f g h i ie ii iə j k kh l m n o oa oo p ph r s t th u uu uə w z ŋ ɑ ɑɑ ɓ ɔ ɔɔ ɗ ə əə ɛ ɛə ɛɛ ɨ ɨə ɨɨ ɲ ʔ


In [10]:
import os
import torch
import soundfile as sf

# 1) Đường dẫn
mel_path = './stableTTS_datasets/mels/0_khm_0308_0011865648.pt'  # đổi sang mel bạn muốn
vocoder_ckpt = './vocoders/pretrained/firefly-gan-base-generator.ckpt'
out_wav = './ffgan_reconstruction.wav'

# 2) Thiết bị và sample rate
from config import MelConfig
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sr = MelConfig().sample_rate  # phải khớp với mel đã trích xuất

# 3) Load vocoder FF-GAN
from api import get_vocoder
vocoder = get_vocoder(vocoder_ckpt, model_name='ffgan').to(device).eval()

# 4) Load mel và suy diễn
mel = torch.load(mel_path, map_location='cpu')            # [n_mels, T]
if mel.dim() == 2:
    mel = mel.unsqueeze(0)                                # [1, n_mels, T]
mel = mel.to(device)

with torch.inference_mode():
    audio = vocoder(mel)                                  # [B, 1, time] hoặc [B, time]
    if audio.dim() == 3:                                  # [1, 1, time] -> [time]
        audio = audio.squeeze(0).squeeze(0)
    elif audio.dim() == 2:                                # [1, time] -> [time]
        audio = audio.squeeze(0)
    audio_np = audio.detach().cpu().numpy()               # float32, shape [time]

# 5) Lưu WAV bằng soundfile
os.makedirs(os.path.dirname(out_wav) or '.', exist_ok=True)
sf.write(out_wav, audio_np, sr, subtype='PCM_16')
print(f'Done. Saved to: {out_wav}')

Done. Saved to: ./ffgan_reconstruction.wav
